<a href="https://colab.research.google.com/github/akhileshmanukonda/bert-model/blob/main/reasearch_ass3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Research and analysis of a Bert pretrained model**

1.   List item
2.   List item


from hugging face analyzing a pretrained bert model.BERT is a very effective tool for text categorisation because of its capacity to comprehend the relationships and context inside text. Through task-specific tweaking of BERT, practitioners may take use of cutting-edge NLP capabilities to get excellent classification accuracy for text data.

In [ ]:
pip install transformers

In [ ]:
pip install torch

In [ ]:
!pip install datasets

here iam starts with importing libraries and writing code for some text classification tasks


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import pipeline
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset

model_name = "bert-base-cased"
# Use AutoModelForSequenceClassification for text classification tasks
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

classifier(
    [
        "jake broke the vase",

    ]
)
sequences = [
    "jake broke the vase\n\nThe vise. This is really a great idea for the Vise\
    I have tried to create and I'm really hoping to see it being incorporated into the Vise."]
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

batch["labels"] = torch.tensor([1])

optimizer = AdamW(model.parameters())
loss = model(**batch).loss
loss.backward()
optimizer.step()


In [ ]:
generator = pipeline("text-generation")
generator("jake broke the vase")

In [ ]:
#load dataset
ds = load_dataset("liuyanchen1015/MULTI_VALUE_mrpc_for_to")
print (ds)

# Load the dataset
dataset = load_dataset("liuyanchen1015/MULTI_VALUE_mrpc_for_to")

# Convert to pandas DataFrame for easier manipulation
train_df = dataset['train'].to_pandas()
test_df = dataset['test'].to_pandas()


In [ ]:
# Display the first few rows of the training set
print(train_df.head())

# Display basic information about the training set
print(train_df.info())

# Check for missing values
print(train_df.isnull().sum())


In [ ]:
# Get descriptive statistics for the dataset
print(train_df.describe())

# Check the distribution of the labels
print(train_df['label'].value_counts())


In [ ]:
# Plot the distribution of the labels
sns.countplot(x='label', data=train_df)
plt.title('Distribution of Labels')
plt.show()


In [ ]:
# Calculate the length of each sentence
train_df['sentence1_length'] = train_df['sentence1'].apply(len)
train_df['sentence2_length'] = train_df['sentence2'].apply(len)

# Plot the distribution of sentence lengths
sns.histplot(train_df['sentence1_length'], kde=True, color='blue', label='Sentence 1 Length')
sns.histplot(train_df['sentence2_length'], kde=True, color='green', label='Sentence 2 Length')
plt.legend()
plt.title('Distribution of Sentence Lengths')
plt.show()


In [ ]:
# Check for duplicate rows
duplicates = train_df.duplicated(subset=['sentence1', 'sentence2']).sum()
print(f"Number of duplicate sentence pairs: {duplicates}")


In [ ]:
from wordcloud import WordCloud

# Generate a word cloud for the first sentences
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(' '.join(train_df['sentence1']))
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud for Sentence 1')
plt.show()

# Generate a word cloud for the second sentences
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(' '.join(train_df['sentence2']))
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud for Sentence 2')
plt.show()



In [ ]:
train_set = ds["train"]
train_set[0]


train_set.features

In [ ]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# Access the correct columns within the train_set Dataset
tokenized_sentences_1 = tokenizer(train_set["sentence1"])
tokenized_sentences_2 = tokenizer(train_set["sentence2"])

In [ ]:
inputs = tokenizer('Around 0335 GMT , Tab shares was up 19 cents , or 4.4 % , at A $ 4.56 , having earlier set a record high of A $ 4.57 .',
 'Tab shares jumped 20 cents , or 4.6 % , to set a record closing high at A $ 4.57 .')
inputs

In [ ]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"])

In [ ]:
tokenized_dataset = tokenizer(
    train_set["sentence1"],
    train_set["sentence2"],
    padding=True,
    truncation=True,
)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

In [ ]:
tokenized_datasets = train_set.map(tokenize_function, batched=True)
tokenized_datasets

In [ ]:
import wandb
wandb.init(project="reasearch_ass3")

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")


model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=3e-5)

In [ ]:
# Assuming 'train_set' is your entire dataset, split it first
from datasets import DatasetDict
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}



# Adjust split ratio as needed
train_testvalid = train_set.train_test_split(test_size=0.2)
train_valid = train_testvalid['test'].train_test_split(test_size=0.5)

# Create a DatasetDict
tokenized_datasets = DatasetDict({
    'train': train_testvalid['train'],
    'validation': train_valid['test'],
    'test': train_valid['train']
})

# Adjust TrainingArguments
training_args = TrainingArguments(
    output_dir="./model_output",
    run_name="unique_run_name",
    save_strategy= "epoch",
    logging_dir="./logs",
    num_train_epochs= 5,
    logging_steps=50,
    learning_rate=2e-5,
    #report_to="wandb",
    )
# Apply tokenization
tokenized_datasets = tokenized_datasets.map(tokenize_function, batched=True)
from transformers import Trainer
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
import numpy as np
trainer.train()


In [ ]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

In [ ]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [ ]:
pip install evaluate

In [ ]:
import evaluate

metric = evaluate.load("f1")
metric.compute(predictions=preds, references=predictions.label_ids)
print(metric)

In [ ]:
# Assuming trainer.state.log_history contains the training history
train_loss = [log['loss'] for log in trainer.state.log_history if 'loss' in log]
# Extract validation loss if available (e.g., using 'eval_loss')
validation_loss = [log['eval_loss'] for log in trainer.state.log_history if 'eval_loss' in log]

# Plotting
plt.figure(figsize=(10, 5))
plt.plot(train_loss, label='Train Loss')
plt.plot(validation_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch/Step')
plt.ylabel('Loss')
plt.legend()
plt.show()